# 🧠 MetaForge: Compare Solvers Colab Demo

This notebook demonstrates how to compare multiple solvers (e.g., Tabu Search, GA, SA, ACO, etc.) across benchmark problems using MetaForge.

---
## ✅ Install MetaForge

In [ ]:
!pip install metaforge

## 📦 Imports

In [ ]:
# from metaforge.utils.compare_solvers import compare_all_benchmarks
from metaforge.utils.plotting import plot_convergence_comparison, plot_solver_summary

In [ ]:
# 🔁 Temporary override of compare_all_benchmarks with URL support

import os
import csv
from metaforge.problems.benchmark_loader import load_job_shop_instance
from metaforge.metaforge_runner import run_solver
from metaforge.utils.timer import Timer

# === Pretty name mapping for solvers ===
pretty_names = {
    "sa": "Simulated Annealing",
    "ts": "Tabu Search",
    "ga": "Genetic Algorithm",
    "aco": "Ant Colony Optimization",
    "q": "Q-Learning",
    "dqn-naive": "DQN (naive)",
    "dqn-replay": "DQN (replay)",
    "neuroevo": "Neuroevolution",
}

def compare_all_benchmarks(
    benchmark_source,
    solvers,
    format="orlib",
    output_csv="results/benchmark_comparison.csv",
    track_schedule=False,
    plot=False
):
    is_url = benchmark_source.startswith("http://") or benchmark_source.startswith("https://")

    if output_csv:
        output_dir = os.path.dirname(output_csv)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)

    benchmark_files = [
        "ft06.txt", "ft10.txt", "ft20.txt",
        "la01.txt", "la02.txt", "la03.txt",
        "la04.txt", "la05.txt"
    ]

    results = []

    for benchmark_file in benchmark_files:
        path = (
            f"{benchmark_source.rstrip('/')}/{benchmark_file}" if is_url
            else os.path.join(benchmark_source, benchmark_file)
        )

        try:
            problem = load_job_shop_instance(path, format=format)
        except Exception as e:
            print(f"⚠️ Failed to load {benchmark_file}: {e}")
            continue

        for solver in solvers:
            solver_label = pretty_names.get(solver, solver)
            print(f"Running {solver_label} on {benchmark_file}...")

            timer = Timer()
            result = run_solver(
                solver,
                problem,
                track_schedule=track_schedule
            )
            elapsed = timer.stop()

            results.append({
                "benchmark": benchmark_file,
                "solver": solver_label,
                "best_score": result["makespan"],
                "runtime_sec": elapsed,
                "best_solution": result["solution"],
                "all_schedules": result["schedules"],
                "history": result["history"],
            })

    # Write summary CSV
    if output_csv:
        with open(output_csv, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["benchmark", "solver", "best_score", "runtime_sec"])
            writer.writeheader()
            for row in results:
                writer.writerow({
                    "benchmark": row["benchmark"],
                    "solver": row["solver"],
                    "best_score": row["best_score"],
                    "runtime_sec": row["runtime_sec"],
                })
        print(f"\n✅ All results saved to {output_csv}")

    # Optional plotting
    if plot:
        from metaforge.utils.visualization import (
            plot_results_from_csv,
            plot_runtime_from_csv,
        )
        plot_results_from_csv(output_csv)
        plot_runtime_from_csv(output_csv)

    return results


## 🚀 Run All Solvers on All Benchmarks

In [ ]:
results = compare_all_benchmarks(
    benchmark_source='https://raw.githubusercontent.com/Mageed-Ghaleb/MetaForge/main/data/benchmarks/',
    solvers=['ts', 'ga', 'sa', 'aco'],
    track_schedule=False,
    plot=True
)

## 📊 Plot Makespan Convergence

In [ ]:
# Pick one benchmark (e.g., ft06.txt)
plot_solver_convergence(results['ft06.txt'])

## 🧠 Compare Solvers Across All Benchmarks

In [ ]:
plot_solver_benchmark_summary(results)